In [3]:
import tensorflow as tf
import larq as lq
import numpy as np
import pandas as pd
import seaborn as sns
from binaryflow import quantizers
from binaryflow.layers import ABCNet,XnorNet,BinaryNet
from binaryflow.block import BiRealNet
from binaryflow.layers.normalization import *
from contextlib import redirect_stdout
import json
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds


2022-08-12 23:12:55.280495: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-12 23:12:55.308164: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-12 23:12:55.308367: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-12 23:12:55.309384: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

## 1. Creating Dataset

In [4]:
import os
import re
import scipy.io

class AudioMNISTRow():
    def __init__(self,data,size:int,rate:int,label,person:str):
        self.data=data
        self.size=size
        self.label=label
        self.person=person
        self.rate=rate
    def __repr__(self)->str:
        return f"{self.person}'s pronunciation for digit {self.label}: {self.data}"
    
    @staticmethod
    def toDataFrame(rows):
        return pd.DataFrame([[row.person,self.rate,row.size,row.data,row.label] for row in rows],columns=["Person","Rate","Size","Data","Label"])
    

def load_audio_mnist(path:str):
    U=[]
    for fileName in os.listdir(path):
        match=re.match(R"([0-9])_(\w+)_[0-9]+\.wav",fileName)
        if match:
            rate,data=scipy.io.wavfile.read(f"{path}/{fileName}")
            size=len(data)
            U.append(AudioMNISTRow(data=data,rate=rate,size=size,label=int(match.group(1)[0]),person=match.group(2)))
    return U


class AudioMNIST(pd.DataFrame):
    def __init__(self,rows):
        super().__init__([[row.person,row.rate,row.size,pd.Series(row.data),row.label] for row in rows],columns=["Person","Rate","Size","Data","Label"])
        self.radius=self["Size"].min()
    
    def augment(self):
        U=[]
        for i in range(self.shape[0]):
            row=self.iloc[i]
            j=i
            while j+self.radius <= len(row["Data"]):
                U.append([row["Person"],row["Rate"],*row["Data"][j:j+self.radius],row["Label"]])
                j+=1
        
        return pd.DataFrame(U,columns=["Person","Rate",*[f"Timestamp {i}" for i in range(self.radius)],"Label"])
    def stochasticCrop(self,count,to_tensor=False,to_list=False,radius=None):
        if radius is None:
            radius=self.radius
        U=[]
        for i in range(self.shape[0]):
            row=self.iloc[i]
            if radius < row["Size"]:
                K=np.random.randint(0,row["Size"]-radius+1,count)
                for k in K:
                    U.append([row["Person"],row["Rate"],*row["Data"][k:k+radius],row["Label"]])
            else:
                K=np.random.randint(0,radius-row["Size"],count)
                n=radius-row["Size"]
                for k in K:
                    U.append([row["Person"],row["Rate"],*np.zeros(k),*row["Data"],*np.zeros(n-k),row["Label"]])
        if to_list:
            return U
        elif to_tensor:
            return tf.constant([V[2:-1] for V in U]),tf.constant([V[-1] for V in U])
        else:
            return pd.DataFrame(U,columns=["Person","Rate",*[f"Timestamp {i}" for i in range(radius)],"Label"])
    def flatten(self):
        L=[]
        for i in range(self.shape[0]):
            row=self.iloc[i]
            for index,s in enumerate(row["Data"]):
                L.append([row["Person"],row["Rate"],row["Label"],index,s])
        return pd.DataFrame(L,columns=["Person","Rate","Label","Timestamp","Amplitude"])
        
def joinSpectralDomain(X):
    shape=X.shape+[1]
    return tf.concat([tf.reshape(X,shape),tf.reshape(tf.signal.dct(X),shape)],axis=-1)

In [5]:
U=load_audio_mnist("dataset/audio-mnist/recordings")
U=AudioMNIST(U)

In [6]:
from sklearn.model_selection import train_test_split
radius=2000
X,y=U.stochasticCrop(10,to_tensor=True,radius=radius)
X=tf.reshape(tf.signal.dct(X),X.shape+[1])
X_train,X_test,y_train,y_test= train_test_split(X.numpy().astype(dtype=float),y.numpy(),train_size=.75)

In [27]:
bnn_kwargs=dict(kernel_quantizer="ste_sign",input_quantizer="ste_sign",kernel_constraint="weight_clip")
abc_kwargs=dict(kernel_quantizers=quantizers.ShiftedSteSign,input_quantizers=quantizers.ShiftedSteSign,kernel_constraint="weight_clip",
               kernel_estimators=3)
Dense=BinaryNet.QuantDense
Conv1D=BinaryNet.QuantConv1D
x=tf.keras.layers.Input(shape=(radius,1))
s=tf.keras.layers.GaussianNoise(stddev=15)(x)
s=s/tf.math.reduce_max(x,axis=-2,keepdims=True)
s=tf.keras.layers.BatchNormalization(momentum=0.999,scale=False)(s)
x1=Conv1D(128,3,activation="tanh",**bnn_kwargs,padding="same",name="hola")(s)
x1=tf.keras.layers.MaxPool1D(3,strides=2)(x1)
x2=tf.keras.layers.BatchNormalization(momentum=0.999,scale=False)(x1)           
x2=Conv1D(128,9,activation="relu",**bnn_kwargs,padding="same")(x1)+x1
x2=tf.keras.layers.MaxPool1D(3,strides=2)(x2)
x2=tf.keras.layers.BatchNormalization(momentum=0.999,scale=False)(x2)            
x3=Conv1D(128,19,activation="relu",**bnn_kwargs,padding="same")(x2)+x2
x3=tf.keras.layers.MaxPool1D(3,strides=2)(x3)
x3=tf.keras.layers.Flatten()(x3)
x3=tf.keras.layers.BatchNormalization(momentum=0.999,scale=False)(x3)
x4=Dense(256,activation="relu",**bnn_kwargs,use_bias=False)(x3)
x4=tf.keras.layers.BatchNormalization(momentum=0.999,scale=False)(x4)
y=tf.keras.layers.Dense(10)(x4)
y=tf.keras.layers.Activation("softmax")(y)
model=tf.keras.Model(x,y)

In [28]:
model.compile(
tf.keras.optimizers.Adam(lr=0.01, decay=0.0001),
loss="categorical_crossentropy",
metrics=["accuracy"],
)


trained_model = model.fit(
    X_train, 
    tf.one_hot(y_train,10),
    batch_size=128, 
    epochs=90,
    validation_data=(X_test,tf.one_hot(y_test,10)),
    shuffle=True
)

/home/ramizouari/anaconda3/envs/tensorflow_2_9/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/90
176/176 [==============================] - 21s 112ms/step - loss: 0.9718 - accuracy: 0.6846 - val_loss: 468.6893 - val_accuracy: 0.0925
Epoch 2/90
176/176 [==============================] - 19s 110ms/step - loss: 0.6908 - accuracy: 0.7695 - val_loss: 168.0598 - val_accuracy: 0.0925
Epoch 3/90
176/176 [==============================] - 20s 111ms/step - loss: 0.5931 - accuracy: 0.7998 - val_loss: 154.5296 - val_accuracy: 0.0925
Epoch 4/90
176/176 [==============================] - 20s 112ms/step - loss: 0.4652 - accuracy: 0.8428 - val_loss: 222.9056 - val_accuracy: 0.0925
Epoch 5/90
176/176 [==============================] - 19s 110ms/step - loss: 0.4471 - accuracy: 0.8482 - val_loss: 10.7429 - val_accuracy: 0.1113
Epoch 6/90
176/176 [==============================] - 19s 110ms/step - loss: 0.3899 - accuracy: 0.8658 - val_loss: 60.1891 - val_accuracy: 0.1008
Epoch 7/90
176/176 [==============================] - 20s 111ms/step - loss: 0.3351 - accuracy: 0.8822 - val_loss: 6.563

In [26]:
trained_model = model.fit(
    X_train, 
    tf.one_hot(y_train,10),
    batch_size=128, 
    epochs=30,
    validation_data=(X_test,tf.one_hot(y_test,10)),
    shuffle=True
)

Epoch 1/30
176/176 [==============================] - 19s 107ms/step - loss: 0.0597 - accuracy: 0.9801 - val_loss: 5.1658 - val_accuracy: 0.4187
Epoch 2/30
176/176 [==============================] - 19s 107ms/step - loss: 0.0442 - accuracy: 0.9848 - val_loss: 6.9967 - val_accuracy: 0.3233
Epoch 3/30
176/176 [==============================] - 19s 106ms/step - loss: 0.0674 - accuracy: 0.9767 - val_loss: 3.3788 - val_accuracy: 0.5156
Epoch 4/30
176/176 [==============================] - 19s 106ms/step - loss: 0.0806 - accuracy: 0.9730 - val_loss: 2.6283 - val_accuracy: 0.4717
Epoch 5/30
176/176 [==============================] - 19s 110ms/step - loss: 0.0402 - accuracy: 0.9864 - val_loss: 5.2961 - val_accuracy: 0.3339
Epoch 6/30
176/176 [==============================] - 19s 106ms/step - loss: 0.0257 - accuracy: 0.9914 - val_loss: 4.6750 - val_accuracy: 0.4173
Epoch 7/30
176/176 [==============================] - 18s 102ms/step - loss: 0.0270 - accuracy: 0.9913 - val_loss: 5.2577 - val_ac

In [14]:
trained_model = model.fit(
    X_train, 
    tf.one_hot(y_train,10),
    batch_size=128, 
    epochs=30,
    validation_data=(X_test,tf.one_hot(y_test,10)),
    shuffle=True
)

Epoch 1/30
176/176 [==============================] - 21s 121ms/step - loss: 0.0409 - accuracy: 0.9863 - val_loss: 2.4476 - val_accuracy: 0.5615
Epoch 2/30
176/176 [==============================] - 21s 120ms/step - loss: 0.0406 - accuracy: 0.9861 - val_loss: 2.5073 - val_accuracy: 0.5884
Epoch 3/30
176/176 [==============================] - 21s 120ms/step - loss: 0.0482 - accuracy: 0.9844 - val_loss: 3.8314 - val_accuracy: 0.4476
Epoch 4/30
176/176 [==============================] - 22s 126ms/step - loss: 0.0502 - accuracy: 0.9826 - val_loss: 3.4743 - val_accuracy: 0.5265
Epoch 5/30
176/176 [==============================] - 22s 125ms/step - loss: 0.0389 - accuracy: 0.9868 - val_loss: 5.0860 - val_accuracy: 0.3981
Epoch 6/30
176/176 [==============================] - 21s 120ms/step - loss: 0.0381 - accuracy: 0.9867 - val_loss: 5.3347 - val_accuracy: 0.3653
Epoch 7/30
176/176 [==============================] - 22s 127ms/step - loss: 0.0413 - accuracy: 0.9860 - val_loss: 4.2507 - val_ac

In [76]:
X_train.max()-X_train.min()

65535

In [103]:
X_train.shape

(22500, 1148, 1)

In [207]:
U["Size"].describe()

count     3000.000000
mean      3499.474667
std       1181.144044
min       1148.000000
25%       2738.750000
50%       3358.500000
75%       4082.250000
max      18262.000000
Name: Size, dtype: float64

In [25]:
json.dump(model.history.history,open("results/AudioMNIST/BinaryNetSpectralOnly2.json","w"))

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2000, 1)]    0           []                               
                                                                                                  
 gaussian_noise (GaussianNoise)  (None, 2000, 1)     0           ['input_1[0][0]']                
                                                                                                  
 tf.math.reduce_max (TFOpLambda  (None, 1, 1)        0           ['input_1[0][0]']                
 )                                                                                                
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 2000, 1)      0           ['gaussian_noise[0][0]',     

In [223]:
U

,Person,Rate,Size,Data,Label
0,george,8000,2384,0 -1489 1 -962 2 -606 3 ...,0
1,george,8000,4727,0 36 1 18 2 63 3 75 4 ...,0
2,george,8000,5958,0 -175 1 -509 2 293 3 24...,0
3,george,8000,3661,0 103 1 138 2 196 3 17...,0
4,george,8000,4050,0 67 1 91 2 104 3 14...,0
...,...,...,...,...,...
2995,yweweler,8000,2877,0 -3 1 -13 2 0 3 -10 4 ...,9
2996,yweweler,8000,2778,0 -7 1 10 2 11 3 12 4 ...,9
2997,yweweler,8000,2815,0 8 1 3 2 8 3 2 4 ...,9
2998,yweweler,8000,3164,0 12 1 4 2 15 3 10 4 ...,9


In [15]:
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 2000, 1)]    0           []                               
                                                                                                  
 gaussian_noise_13 (GaussianNoi  (None, 2000, 1)     0           ['input_14[0][0]']               
 se)                                                                                              
                                                                                                  
 tf.math.reduce_max (TFOpLambda  (None, 1, 1)        0           ['input_14[0][0]']               
 )                                                                                                
                                                                                            

In [32]:
U["Data"].map(lambda D:D.max())

0       10354
1        7605
2        8221
3        6404
4        6165
        ...  
2995      820
2996     1880
2997      978
2998      966
2999      951
Name: Data, Length: 3000, dtype: int16

In [38]:
L=U.copy()

In [40]:
L["Mean"]=U["Data"].map(lambda D:D.mean())
L["Max"]=U["Data"].map(lambda D:D.max())
L["Min"]=U["Data"].map(lambda D:D.min())
L["Std"]=U["Data"].map(lambda D:D.std())

In [45]:
L.groupby("Person").mean()

,Rate,Size,Label,Mean,Max,Min,Std
Person,,,,,,,
george,8000.0,3533.740,4.5,-0.850576,9715.212,-10715.504,1986.108635
jackson,8000.0,4131.680,4.5,-0.062773,13397.494,-14633.924,2909.242220
lucas,8000.0,4593.688,4.5,0.920933,10756.182,-16556.192,2039.574311
nicolas,8000.0,2793.502,4.5,-237.696940,5970.432,-8285.184,1588.342554
theo,8000.0,3110.898,4.5,-0.004797,1816.584,-1487.426,367.106162
yweweler,8000.0,2833.340,4.5,-0.411068,1585.138,-2448.024,398.944125


In [49]:
X_train.shape

(22500, 2000, 1)

In [51]:
len(model.history.history["loss"])

30

In [27]:
X

<tf.Tensor: shape=(30000, 4000), dtype=float32, numpy=
array([[-2352.      , -2022.      , -5387.      , ...,  -189.77325 ,
          -39.983086,   264.06302 ],
       [ 3530.      ,  3051.      ,  2195.      , ...,   200.67476 ,
           95.373924,  -260.94455 ],
       [-2318.      , -1776.      , -2747.      , ...,   363.40054 ,
          388.05865 ,  -143.78802 ],
       ...,
       [ -344.      ,  -235.      ,   -95.      , ...,   302.15637 ,
           32.97239 ,    63.057533],
       [  128.      ,   113.      ,    95.      , ...,  -188.84828 ,
         -389.05484 ,  -243.38365 ],
       [   51.      ,    82.      ,    75.      , ...,  -301.0248  ,
          -50.201347,   -66.22906 ]], dtype=float32)>

In [15]:
model.save("trained/ABCNet-AudioMNIST")

INFO:tensorflow:Assets written to: trained/ABCNet-AudioMNIST/assets


INFO:tensorflow:Assets written to: trained/ABCNet-AudioMNIST/assets


In [16]:
lq.models.summary(model)

+model_1 stats-----------------------------------------------------------------------------------------------+
| Layer                   Input prec.          Outputs   # 1-bit  # 32-bit   Memory  1-bit MACs  32-bit MACs |
|                               (bit)                        x 1       x 1     (kB)                          |
+------------------------------------------------------------------------------------------------------------+
| input_4                           -    (-1, 2000, 1)         0         0        0           ?            ? |
| gaussian_noise_3                  -    (-1, 2000, 1)         0         0        0           ?            ? |
| tf.math.reduce_max_3              -       (-1, 1, 1)         0         0        0           ?            ? |
| tf.math.truediv_3                 -    (-1, 2000, 1)         0         0        0           ?            ? |
| batch_normalization_13            -    (-1, 2000, 1)         0         2     0.01           0            0 |
|

In [7]:
model.layers

NameError: name 'model' is not defined

In [10]:
tf.ones(shape=(3,3,3))

<tf.Tensor: shape=(3, 3, 3), dtype=float32, numpy=
array([[[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]], dtype=float32)>

In [9]:
p=tf.keras.layers.Input(shape=(11,))
q=ABCNet.ABCDense(10,**abc_kwargs)(p)
M=tf.keras.Model(p,q)

a
44
33
44


TypeError: Keras symbolic inputs/outputs do not implement `__len__`. You may be trying to pass Keras symbolic inputs/outputs to a TF API that does not register dispatching, preventing Keras from automatically converting the API call to a lambda layer in the Functional Model. This error will also get raised if you try asserting a symbolic input/output directly.